In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN を使ったテキスト分類

In [3]:
# !pip install -q tf-nightly
# import tensorflow_datasets as tfds
# !pip3 list | grep tensorflow
# import tensorflow as tf

# !pip3 install sentencepiece
import sentencepiece as spm

import json
import os
import pandas as pd

In [4]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#   plt.plot(history.history[metric])
#   plt.plot(history.history['val_'+metric], '')
#   plt.xlabel("Epochs")
#   plt.ylabel(metric)
#   plt.legend([metric, 'val_'+metric])
#   plt.show()

## Data


### load

In [5]:
sbj_name = 'Eigo'
attr_name = 'answer_type'

attr_csv_path = f'../{sbj_name}_{attr_name}_ds.tsv'
df = pd.read_csv(attr_csv_path, delimiter='\t')

df

,Unnamed: 0,answer_type,<instruction/>,contents
0,0,(symbol-sentence)*2,<instruction>次の問い(問１・問２)において，下線部(a)・(b)の単語のアクセ...,<label><label>問１<label/>問１<label/><ansColumn><...
1,1,(symbol-sentence)*2,<instruction>次の問い(問１・問２)において，下線部(a)・(b)の単語のアクセ...,<label><label>問２<label/>問２<label/><ansColumn><...
2,2,sentence,<instruction>次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～...,<label><label>問１<label/>問１<label/><ansColumn><...
3,3,sentence,<instruction>次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～...,<label><label>問２<label/>問２<label/><ansColumn><...
4,4,sentence,<instruction>次の会話の下線部(1)～(4)について，それぞれ以下の問い(問１～...,<label><label>問３<label/>問３<label/><ansColumn><...
...,...,...,...,...
978,978,sentence,<instruction>次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なもの...,<label><label>問２<label/>問２<label/><data><ref><...
979,979,sentence,<instruction>次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なもの...,<label><label>問３<label/>問３<label/><data><ref><...
980,980,sentence,<instruction>次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なもの...,<label><label>問４<label/>問４<label/><data><ref><...
981,981,sentence,<instruction>次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なもの...,<label><label>問５<label/>問５<label/><data><data>...


### Tokenize
SentencePiece を使用。
- タグ あり／なし

In [6]:
from sklearn.model_selection import train_test_split

train_examples, test_examples = train_test_split(
                                    df, test_size=0.2, random_state=0)
train_examples.head(5)

,Unnamed: 0,answer_type,<instruction/>,contents
893,893,sentence,<instruction>次の問い（問１～４）において，第一アクセント（第一強勢）の位置がほ...,<label><label>問２<label/>問２<label/><ansColumn><...
158,158,sentence,<instruction>次の広告に関する以下の問い(問１～３)を読み， 39 ～ 41 に...,<label><label>問２<label/>問２<label/><data><ansCo...
154,154,sentence,<instruction>次の英文は，体育の授業時間数について，クラスで行われたディスカッシ...,<label><label>Ｃ<label/>Ｃ<label/><instruction><...
40,40,sentence,<instruction>次の問い(問１～５)に対する答えとして最も適当なものを，それぞれ以...,<label><label>Ｂ<label/>Ｂ<label/><instruction><...
793,793,sentence,<instruction>次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なもの...,<label><label>問５<label/>問５<label/><data><data>...


In [7]:
df_tmp = df[['<instruction/>', 'contents']]
df_tmp['<instruction/>'][6]

'<instruction>次の問い(問１～10)の 7 ～ 16 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。<instruction/>'

In [9]:
m_dir = 'model/SentencePiece'
os.makedirs(m_dir, exist_ok=True)
df.to_csv(f'{m_dir}/tmp.txt', sep='\t')

# arg_str = '--input={m_dir}/tmp.txt --model_prefix={m_dir}/m_user ' + '--user_defined_symbols=<sep>,<cls>' + ',<ansColumn/>,<label>' + ' --vocab_size=2000'
# spm.SentencePieceTrainer.train(arg_str)

spm.SentencePieceTrainer.train(f'--input={m_dir}/tmp.txt --model_prefix={m_dir}/m  --user_defined_symbols=<sep>,<cls>,<pad>   --vocab_size=2000')
sp = spm.SentencePieceProcessor()  # model_file='SentencePiece/test_model.model'

sp.load(f'{m_dir}/m.model')

True

In [10]:
# encode: text => id
tokenized_tokens =  sp.encode_as_pieces('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_tokens)

tokenized_ids = sp.encode_as_ids('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_ids)

decoded_text = sp.decode(tokenized_ids)
print(decoded_text)

['▁', '次の', '問', 'い', '(', '問', '1～3)', 'の会話の', '▁17', '▁～', '▁19', '▁に入れる', 'の', 'に', '最も適当な', 'ものを', ',', 'それぞれ以下の', '1～4', 'のうちから一つ', 'ず', 'つ選べ', '。']
[39, 49, 29, 52, 56, 29, 132, 323, 387, 153, 479, 144, 55, 124, 128, 94, 34, 173, 83, 90, 89, 86, 58]
次の問い(問1～3)の会話の 17 ～ 19 に入れるのに最も適当なものを,それぞれ以下の1～4のうちから一つずつ選べ。


In [11]:
example_content = df_tmp['contents'][20]
print(example_content, sp.encode_as_pieces(example_content))

<label><label>問２<label/>問２<label/><data><blank><blank><blank/><blank/><ansColumn><ansColumn>22<ansColumn/>22<ansColumn/><blank><blank><blank/><blank/><ansColumn><ansColumn>23<ansColumn/>23<ansColumn/><blank><blank><blank/><blank/><data><blank><blank><blank/><blank/><ansColumn><ansColumn>22<ansColumn/>22<ansColumn/><blank><blank><blank/><blank/><ansColumn><ansColumn>23<ansColumn/>23<ansColumn/><blank><blank><blank/><blank/>New information about diet<data/>New information about diet<data/><choices><choice><cNum><cNum>①<cNum/>①<cNum/><choice><cNum><cNum>①<cNum/>①<cNum/><choice/><choice/><choice><cNum><cNum>②<cNum/>②<cNum/><choice><cNum><cNum>②<cNum/>②<cNum/><choice/><choice/><choice><cNum><cNum>③<cNum/>③<cNum/><choice><cNum><cNum>③<cNum/>③<cNum/><choice/><choice/><choice><cNum><cNum>④<cNum/>④<cNum/><choice><cNum><cNum>④<cNum/>④<cNum/><choice/><choice/><choice><cNum><cNum>⑤<cNum/>⑤<cNum/><choice><cNum><cNum>⑤<cNum/>⑤<cNum/><choice/><choice/><choices><choice><cNum><cNum>①<cNum/>①<cNum/><cho

In [26]:
# for index in encoded_string:
#   print('{} ----> {}'.format(index, encoder.decode([index])))

## 訓練用データの準備

In [32]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for instruction in df['<instruction/>']:
    tokens = sp.encode_as_pieces(instruction)
    for word in tokens:
        if word in word2index: continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))


NameError: name 'datasets' is not defined

### 

In [14]:
## 系列の長さを揃えてバッチでまとめる

from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

def sentence2index(sentence):
    wakati = make_wakati(sentence)
    return [word2index[w] for w in wakati]

def category2index(cat):
    return [cat2index[cat]]

index_datasets_title_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for title, category in zip(datasets["title"], datasets["category"]):
  index_title = sentence2index(title)
  index_category = category2index(category)
  index_datasets_title_tmp.append(index_title)
  index_datasets_category.append(index_category)
  if max_len < len(index_title):
    max_len = len(index_title)

# 系列の長さを揃えるために短い系列にパディングを追加
# 後ろパディングだと正しく学習できなかったので、前パディング
index_datasets_title = []
for title in index_datasets_title_tmp:
  for i in range(max_len - len(title)):
    title.insert(0, 0) # 前パディング
#     title.append(0)　# 後ろパディング
  index_datasets_title.append(title)

train_x, test_x, train_y, test_y = train_test_split(index_datasets_title, index_datasets_category, train_size=0.7)

# データをバッチでまとめるための関数
def train2batch(title, category, batch_size=100):
  title_batch = []
  category_batch = []
  title_shuffle, category_shuffle = shuffle(title, category)
  for i in range(0, len(title), batch_size):
    title_batch.append(title_shuffle[i:i+batch_size])
    category_batch.append(category_shuffle[i:i+batch_size])
  return title_batch, category_batch

## モデルの作成

In [15]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'tf' is not defined

## モデルの訓練

In [16]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores

# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)

<class 'tensorflow.python.data.ops.dataset_ops.PaddedBatchDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.PaddedBatchDataset'>


In [18]:
test_num = len(test_x)
a = 0
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
print("predict : ", a / test_num)
# predict :  0.6967916854948034

391/391 [==============================] - 28s 72ms/step - loss: 0.6909 - accuracy: 0.5000
Test Loss: 0.6909476518630981
Test Accuracy: 0.5


## 2つ以上の LSTM レイヤーを重ねる

Keras のリカレントレイヤーには、コンストラクタの `return_sequences` 引数でコントロールされる2つのモードがあります。

* それぞれのタイムステップの連続した出力のシーケンス全体（shape が `(batch_size, timesteps, output_features)` の3階テンソル）を返す。
* それぞれの入力シーケンスの最後の出力だけ（shape が `(batch_size, output_features)` の2階テンソル）を返す。

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [25]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [26]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
309/391 [======================>.......] - ETA: 24s - loss: 0.6921 - accuracy: 0.4997

KeyboardInterrupt: 

In [27]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 50s 129ms/step - loss: 0.6772 - accuracy: 0.5000
Test Loss: 0.67717444896698
Test Accuracy: 0.5


In [28]:
# パディングなしのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print(predictions)

[[-0.0279653]]


In [29]:
# パディングありのサンプルテキストの推論

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)

[[-0.16973719]]


In [30]:
plot_graphs(history, 'accuracy')

NameError: name 'history' is not defined

In [ ]:
plot_graphs(history, 'loss')

[GRU レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)など既存のほかのレイヤーを調べてみましょう。

カスタム RNN の構築に興味があるのであれば、[Keras RNN ガイド](../../guide/keras/rnn.ipynb) を参照してください。